# Steps
1. Get dataset of 20 stock from database (.csv)
2. 

# Further ideas
1. Give multiple subsets of stocks (e.g. "Becky" as a strategy)


# Selected stocks
20 stocks were randomly selected from the S&P 500 pool

In [10]:
stock_symbols = ['MCD', 'ESS', 'ABC', 'PFE', 'MA', 'GWW', 'AOS', 'GD', 'YUM', 'LOW', 'ALB', 'IFF', 'TPR', 'CB', 'ANTM', 'WAT', 'CPB', 'IPGP', 'SBUX', 'HSY']

## Imports

In [1]:
import requests
import os
import pandas as pd
import time
from alpha_vantage.timeseries import TimeSeries

## Getting the data

In [2]:
API_KEY = "1077F37TLBGNSVY2"

In [ ]:
full_data = []

for symbol in stock_symbols:
    query_params["symbol"] = symbol
    resp = requests.get(url=url, params=query_params)
    resp.raise_for_status()

In [3]:
STANDARD_DELAY = 60 // 5
DATA_DIR_PATH = r"C:\Users\Jonas\Desktop\CGI\dev\exp\data"
ts = TimeSeries(key=API_KEY, output_format="pandas")

CSV_COLS = ["symbol", "date", "1. open", "2. high", "3. low", "4. close", "5. adjusted close",
           "6. volume", "7. dividend amount", "8. split coefficient"]

### Writing from av API to .csv files

In [11]:
master_df = pd.DataFrame(columns=CSV_COLS)

for idx, symbol in enumerate(stock_symbols):
#     file_path = os.path.join(DATA_DIR_PATH, f"{symbol}.csv")
    df, _ = ts.get_daily_adjusted(symbol=symbol, outputsize="full")
    df["symbol"] = symbol
    master_df.append(other=df)
    print(f"Appended stock {idx}: <{symbol}>. Waiting...")
    time.sleep(STANDARD_DELAY)

Appended stock MCD. Waiting...
Appended stock ESS. Waiting...
Appended stock ABC. Waiting...
Appended stock PFE. Waiting...
Appended stock MA. Waiting...
Appended stock GWW. Waiting...
Appended stock AOS. Waiting...
Appended stock GD. Waiting...
Appended stock YUM. Waiting...
Appended stock LOW. Waiting...
Appended stock ALB. Waiting...
Appended stock IFF. Waiting...
Appended stock TPR. Waiting...
Appended stock CB. Waiting...
Appended stock ANTM. Waiting...
Appended stock WAT. Waiting...
Appended stock CPB. Waiting...
Appended stock IPGP. Waiting...
Appended stock SBUX. Waiting...
Appended stock HSY. Waiting...


### Merging all .csv files

In [4]:
file_names = [os.path.join(DATA_DIR_PATH, f) for f in os.listdir(DATA_DIR_PATH) if os.path.isfile(os.path.join(DATA_DIR_PATH, f))]

In [5]:
df_list = []
for f in file_names:
    df = pd.read_csv(f, index_col=None, header=0)
    df['symbol'] = f.split("\\")[-1].split(".")[0]
    df_list.append(df)
    
df = pd.concat(df_list, axis=0, ignore_index=True)

### Preprocessing

In [45]:
PRICE_COLUMNS = [
    'date',
    'open',
    'high',
    'low',
    'close',
    'adjusted_close',
    'volume',
    'dividend_amount',
    'split_coefficient',
    'symbol'
]

In [40]:
cols_to_rename = {}
for c in df.columns:
    cols_to_rename[c] = "".join(list(filter(lambda s: s.isalpha() or s == " ", c))).strip()
    cols_to_rename[c] = cols_to_rename[c].replace(" ", "_")
    
df.rename(columns=cols_to_rename, inplace=True)

In [109]:
df['date'] = pd.to_datetime(df['date'])
df['symbol'] = df['symbol'].astype("string") # even though this doesn't do anything
df.dtypes

date                 datetime64[ns]
open                        float64
high                        float64
low                         float64
close                       float64
adjusted_close              float64
volume                      float64
dividend_amount             float64
split_coefficient           float64
symbol                       string
dtype: object

# Algorithm

## Class definitions

In [118]:
class Market:
    def __init__(self, prices: pd.DataFrame, name: str = ""):
        assert all([c in PRICE_COLUMNS for c in prices.columns]), \
            "Wrong columns!"
        self.prices = prices
        # sort by date ascending, then by symbol descending
        self.prices.sort_values(by=["date", "symbol"], inplace=True, ascending=[True, False])
        self.name = name or "default_market"
    
    @property
    def symbols(self):
        return list(self.prices.symbol.unique())
    
    def __iter__(self):
        self.curr = 1
        return self
    
    def __len__(self):
        return len(self.prices)
    
    def __next__(self) -> pd.DataFrame:
        if self.curr <= len(self.prices):
            self.max_date = self.prices.date.sort_values().unique()[self.curr - 1]
            state = self.prices.loc[self.prices.date <= self.max_date]
            self.curr += 1
            return state
        else:
            raise StopIteration
    
    def __str__(self):
        return f"Market [{self.name}] (current market_date: {self.max_date})"

#### Calculation of dividend yield
__On certain date (choose 01/02):__
1. Get the last paid dividend amount from prev. year := `dividend_payment`
2. Multiply by the number of times dividends were paid last year (mostly: 4) := `dividend_total`
3. divide `dividend_total` by `current_share_price`

In [122]:
from collections import namedtuple

Transaction = namedtuple(
    "Transaction", 
    field_names=["symbol", "amount", "date", "stock_price", "total_value"]
)

In [56]:
class Agent:
    def __init__(self, name: str, starting_capital: float, market: Market):
        self.name = name
        self.cash = starting_capital
        self.market = market
        self.trading_history = []

In [127]:
t = Transaction(symbol="AAPL", amount=10, date="2020-01-01", stock_price=3000.00, total_value=30_000.00)

In [128]:
t.amount

10